In [ ]:
#URL downloader:

import requests
from bs4 import BeautifulSoup
from time import sleep

class URLDownloader:
    def __init__(self):
        self.urls = []
        self.results = {}

    def download_and_parse(self, retries=3, delay=2):
        for url in self.urls:
            for attempt in range(retries):
                try:
                    response = requests.get(url)
                    response.raise_for_status()
                    soup = BeautifulSoup(response.content, 'html.parser')
                    self.results[url] = soup  
                    break  
                except requests.exceptions.HTTPError as http_err:
                    error_message = f"HTTP error occurred: {http_err}"
                except requests.exceptions.ConnectionError as conn_err:
                    error_message = f"Connection error occurred: {conn_err}"
                except requests.exceptions.Timeout as timeout_err:
                    error_message = f"Timeout error occurred: {timeout_err}"
                except requests.exceptions.RequestException as req_err:
                    error_message = f"General error occurred: {req_err}"

                if attempt < retries - 1:  
                    sleep(delay)
                else:  
                    self.results[url] = error_message

    def get_user_urls(self):
        try:
            num_urls = int(input("Enter the number of URLs you want to add: "))
            if num_urls <= 0:
                print("Please enter a positive number.")
                return
            
            print(f"Enter {num_urls} URLs (one per line):")
            for _ in range(num_urls):
                url = input().strip()
                if url:
                    self.urls.append(url)
        except ValueError:
            print("Invalid input. Please enter a valid number.")

    def print_results(self):
        print("\nResults:")
        for url, result in self.results.items():
            if isinstance(result, BeautifulSoup):
                title = result.title.string if result.title else "No title found"
                print(f"URL: {url}\nTitle: {title}\n")
            else:
                print(f"URL: {url}\nError: {result}\n")

if __name__ == "__main__":
    downloader = URLDownloader()
    print("Enter URLs to download:")
    downloader.get_user_urls()

    if downloader.urls:
        downloader.download_and_parse()
        downloader.print_results()
    else:
        print("No URLs entered.")
